In [1]:
import os
import os.path as osp
import pandas as pd
import SQUIDp.util as sqd
import numpy as np
import sys
from skimage.measure import regionprops
# from SQUIDp.data import get_cell_ids_in_patch, match_patch_id_to_expr, match_patch_id_to_PIL, plots_n_visualizations
from hest import iter_hest
import random
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt

In [2]:
hest_data_dir = "./raw_HEST_data/"
meta_df = pd.read_csv("hf://datasets/MahmoodLab/hest/HEST_v1_1_0.csv")

In [3]:
meta_df

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries,hest_version_added
0,Fresh Frozen Mouse Brain Hemisphere with 5K Mo...,TENX159,TENX159.tif,Brain,Healthy,NaN,Mus musculus,NaN,Xenium,7/31/24,...,NaN,0.273768,0.274027,40x,17051,24689,Brain,NaN,NaN,v1_1_0
1,FFPE Human Skin Primary Dermal Melanoma with 5...,TENX158,TENX158.tif,Skin,Cancer,SKCM,Homo sapiens,NaN,Xenium,7/31/24,...,NaN,0.273777,0.273754,40x,18669,35787,Skin,Primary Dermal Melanoma,NaN,v1_1_0
2,FFPE Human Prostate Adenocarcinoma with 5K Hum...,TENX157,TENX157.tif,Prostate,Cancer,PRAD,Homo sapiens,NaN,Xenium,7/31/24,...,NaN,0.273772,0.273741,40x,25002,49976,Prostate,NaN,NaN,v1_1_0
3,Characterization of immune cell populations in...,TENX156,TENX156.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Visium HD,7/11/24,...,NaN,0.264583,0.273802,40x,71106,58791,Colon,Stage II-A,"Visium HD, Sample P1 CRC",v1_1_0
4,Characterization of immune cell populations in...,TENX155,TENX155.tif,Bowel,Cancer,COAD,Homo sapiens,Patient 1,Visium HD,7/11/24,...,NaN,0.264583,0.273874,40x,75250,48740,Colon,NaN,"Visium HD, Sample P2 CRC",v1_1_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,spatialLIBD,MISC5,MISC5.tif,Brain,Healthy,NaN,Homo sapiens,,Visium,NaN,...,NaN,NaN,0.727113,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151672,v1_0_0
1225,spatialLIBD,MISC4,MISC4.tif,Brain,Healthy,NaN,Homo sapiens,,Visium,NaN,...,NaN,NaN,0.726109,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151673,v1_0_0
1226,spatialLIBD,MISC3,MISC3.tif,Brain,Healthy,NaN,Homo sapiens,,Visium,NaN,...,NaN,NaN,0.725124,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151674,v1_0_0
1227,spatialLIBD,MISC2,MISC2.tif,Brain,Healthy,NaN,Homo sapiens,,Visium,NaN,...,NaN,NaN,0.726109,20x,13332,13332,dorsolateral prefrontal cortex,NaN,151675,v1_0_0


In [10]:
import os
import numpy as np
from scipy.stats import pearsonr

# --- Compute Pearson correlation ---
def pearson_corr(y_true, y_pred):
    corrs = []
    for i in range(y_true.shape[1]):
        std_true = np.std(y_true[:, i])
        std_pred = np.std(y_pred[:, i])
        if std_true == 0 or std_pred == 0:
            corr = np.nan
        else:
            corr, _ = pearsonr(y_true[:, i], y_pred[:, i])
        corrs.append(corr)
    return np.nanmean(corrs), corrs

# --- Directory and file list ---
embed_dir = "./virchow_eval_outputs"
embedding_files = [f for f in os.listdir(embed_dir) if f.endswith("_predicted_expression.npy")]

if not embedding_files:
    raise RuntimeError("No predicted expression files found in the specified directory.")

performance_dict = {}

# --- Loop over all prediction files ---
for pred_file in embedding_files:
    # Extract base ID
    base = pred_file.replace("_predicted_expression.npy", "")
    pred_path = os.path.join(embed_dir, pred_file)

    # Load predictions
    y_pred = np.load(pred_path)

    # Load corresponding metadata to get true expression
    # (Assume your original metadata files are somewhere accessible)
    meta_path = os.path.join("./virchow_embeds", f"{base}_metadata.npy")
    if not os.path.exists(meta_path):
        print(f"⚠️ Missing metadata for {base}, skipping.")
        continue

    metadata = np.load(meta_path, allow_pickle=True)

    # Recreate the padded true expression matrix
    max_len = max(len(m[3]) for m in metadata)
    y_true = np.stack([
        np.pad(m[3], (0, max_len - len(m[3])), mode="constant")
        for m in metadata
    ])

    # Sanity check
    if y_pred.shape != y_true.shape:
        print(f"⚠️ Shape mismatch in {base}: pred shape {y_pred.shape}, true shape {y_true.shape}")
        continue

    # Compute mean Pearson correlation
    mean_pearson, _ = pearson_corr(y_true, y_pred)

    performance_dict[base] = round(mean_pearson, 4)

# --- Summary output ---
print("\n🎯 Final performance summary:")
for k, v in performance_dict.items():
    print(f"  {k}: mean Pearson correlation = {v}")



🎯 Final performance summary:
  patch_to_expr_TENX119: mean Pearson correlation = 0.6954
  patch_to_expr_TENX114: mean Pearson correlation = 0.8513
  patch_to_expr_TENX111: mean Pearson correlation = 0.8194
  patch_to_expr_TENX96: mean Pearson correlation = 0.821
  patch_to_expr_TENX122: mean Pearson correlation = 0.8063
  patch_to_expr_NCBI784: mean Pearson correlation = 0.8056
  patch_to_expr_TENX148: mean Pearson correlation = 0.8203
  patch_to_expr_TENX94: mean Pearson correlation = 0.8441
  patch_to_expr_TENX157: mean Pearson correlation = 0.2493
  patch_to_expr_TENX106: mean Pearson correlation = 0.7637
  patch_to_expr_TENX147: mean Pearson correlation = 0.7543
  patch_to_expr_TENX126: mean Pearson correlation = 0.7266
  patch_to_expr_TENX132: mean Pearson correlation = 0.6031
  patch_to_expr_TENX121: mean Pearson correlation = 0.7816
  patch_to_expr_TENX158: mean Pearson correlation = 0.6478
  patch_to_expr_TENX98: mean Pearson correlation = 0.8027
  patch_to_expr_TENX97: mean P

In [14]:
print(performance_dict)

{'patch_to_expr_TENX119': 0.6954, 'patch_to_expr_TENX114': 0.8513, 'patch_to_expr_TENX111': 0.8194, 'patch_to_expr_TENX96': 0.821, 'patch_to_expr_TENX122': 0.8063, 'patch_to_expr_NCBI784': 0.8056, 'patch_to_expr_TENX148': 0.8203, 'patch_to_expr_TENX94': 0.8441, 'patch_to_expr_TENX157': 0.2493, 'patch_to_expr_TENX106': 0.7637, 'patch_to_expr_TENX147': 0.7543, 'patch_to_expr_TENX126': 0.7266, 'patch_to_expr_TENX132': 0.6031, 'patch_to_expr_TENX121': 0.7816, 'patch_to_expr_TENX158': 0.6478, 'patch_to_expr_TENX98': 0.8027, 'patch_to_expr_TENX97': 0.8493, 'patch_to_expr_TENX140': 0.8586, 'patch_to_expr_TENX139': 0.7289, 'patch_to_expr_TENX116': 0.4428, 'patch_to_expr_TENX120': 0.8582, 'patch_to_expr_TENX105': 0.7409, 'patch_to_expr_TENX115': 0.8896, 'patch_to_expr_TENX138': 0.8136, 'patch_to_expr_TENX149': 0.8249, 'patch_to_expr_TENX134': 0.8076, 'patch_to_expr_NCBI785': 0.7811, 'patch_to_expr_TENX99': 0.7584, 'patch_to_expr_TENX133': 0.644, 'patch_to_expr_TENX117': 0.8419, 'patch_to_expr_T

In [16]:
# Cleaned dict: remove "patch_to_expr_" prefix
performance_dict = {
    k.replace("patch_to_expr_", ""): v
    for k, v in performance_dict.items()
}

# Show result
print(performance_dict)

{'TENX119': 0.6954, 'TENX114': 0.8513, 'TENX111': 0.8194, 'TENX96': 0.821, 'TENX122': 0.8063, 'NCBI784': 0.8056, 'TENX148': 0.8203, 'TENX94': 0.8441, 'TENX157': 0.2493, 'TENX106': 0.7637, 'TENX147': 0.7543, 'TENX126': 0.7266, 'TENX132': 0.6031, 'TENX121': 0.7816, 'TENX158': 0.6478, 'TENX98': 0.8027, 'TENX97': 0.8493, 'TENX140': 0.8586, 'TENX139': 0.7289, 'TENX116': 0.4428, 'TENX120': 0.8582, 'TENX105': 0.7409, 'TENX115': 0.8896, 'TENX138': 0.8136, 'TENX149': 0.8249, 'TENX134': 0.8076, 'NCBI785': 0.7811, 'TENX99': 0.7584, 'TENX133': 0.644, 'TENX117': 0.8419, 'TENX124': 0.8585, 'TENX142': 0.6545, 'TENX125': 0.8168, 'NCBI783': 0.8474, 'TENX95': 0.8327, 'TENX123': 0.8515}


In [18]:
import pandas as pd

# Load your meta_df
# meta_df = pd.read_csv(...)  # or however you loaded it

# 3. Make sure "organ_oncotree_code" is defined
meta_df["organ_oncotree_code"] = meta_df["organ"] + "_" + meta_df["oncotree_code"].fillna("NA")

# 4. Keep only rows whose id is in performance_dict
meta_filtered = meta_df[meta_df["id"].isin(performance_dict.keys())].copy()

# 5. Map performance
meta_filtered["performance"] = meta_filtered["id"].map(performance_dict)

# 6. Group by organ_oncotree_code and average
summary = (
    meta_filtered.groupby("organ_oncotree_code")["performance"]
    .mean()
    .reset_index()
    .sort_values("organ_oncotree_code")
)

# 7. Create a single-row DataFrame
row_df = pd.DataFrame([summary["performance"].values], columns=summary["organ_oncotree_code"].values)

# Optional: Add model name column
row_df.insert(0, "Model", "Virchow2")

# 8. Show final table
print(row_df)

      Model  Bone_NA  Bowel_COAD  Bowel_COADREAD  Bowel_NA  Brain_GBM  \
0  Virchow2   0.6031    0.804725          0.7289    0.8513     0.8136   

   Breast_IDC  Breast_ILC  Heart_NA  Kidney_NA  Kidney_PRCC  Liver_HCC  \
0    0.811029     0.83255    0.6954     0.7637       0.7409     0.8582   

   Liver_NA  Lymphoid_ALL  Lymphoid_NA  Ovary_HGSOC  Pancreas_PAAD  \
0    0.7816        0.8076       0.7731       0.6545          0.676   

   Prostate_PRAD  Skin_NA  Skin_SKCM  
0         0.2493   0.8289     0.7931  


In [19]:
print(row_df.to_string(index=False))


   Model  Bone_NA  Bowel_COAD  Bowel_COADREAD  Bowel_NA  Brain_GBM  Breast_IDC  Breast_ILC  Heart_NA  Kidney_NA  Kidney_PRCC  Liver_HCC  Liver_NA  Lymphoid_ALL  Lymphoid_NA  Ovary_HGSOC  Pancreas_PAAD  Prostate_PRAD  Skin_NA  Skin_SKCM
Virchow2   0.6031    0.804725          0.7289    0.8513     0.8136    0.811029     0.83255    0.6954     0.7637       0.7409     0.8582    0.7816        0.8076       0.7731       0.6545          0.676         0.2493   0.8289     0.7931


In [13]:
import numpy as np

# Path to one of your predicted expression files
file_path = "./virchow_eval_outputs/patch_to_expr_TENX158_predicted_expression.npy"

# Load the array
data = np.load(file_path)

# Print basic info
print("✅ Loaded array:")
print("Shape:", data.shape)
print("Data type:", data.dtype)

# Optionally print the entire array (be careful if it's large!)
print("\nContents:\n", data)


✅ Loaded array:
Shape: (2179, 10017)
Data type: float32

Contents:
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02257455 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
